# Biogen - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
#import consts

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("EFPIA_Disclosure_Report_CH_2019.pdf", pages='all', lattice=True)

## Format Table

In [9]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2', 'empty3']

#Remove "HCP" & "HCO"
df_export.loc[df_export['name'].str.contains("HCP", na=False), 'name'] = np.NaN
df_export.loc[df_export['name'].str.contains("HCO", na=False), 'name'] = np.NaN
df_export.loc[df_export['name'].str.contains("R&D", na=False), 'name'] = np.NaN

#Shift all where empty not empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Copy empty1 to total, if not empty
df_export.loc[df_export['empty1'].notna(), 'total'] = df_export['empty1']

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty
df_export.drop(columns=['empty1', 'empty2', 'empty3'], inplace=True)

#Remove Duplicate
df_export = df_export.drop_duplicates()

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Set Type
df_export = set_type_by_alphabetical_order(df_export)

#Remove Carrination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'biogen')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)